In [84]:
from torchvision.models import resnet18, ResNet18_Weights

# Load a pretrained model using the new API
weights = ResNet18_Weights.DEFAULT
model = resnet18(weights=weights)


In [85]:
#Freeze all layers
for param in model.parameters():
    param.requires_grad = False

In [86]:

#Unfreeze the last two layers
for param in model.layer4.parameters():
    param.requires_grad = True

for param in model.fc.parameters():
    param.requires_grad = True


In [87]:

import torch.nn as nn

num_classes = 10

model.fc = nn.Linear(model.fc.in_features, num_classes)

In [88]:
# Train the last layer
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD([
    {"params": model.layer4.parameters()},
    {"params": model.fc.parameters()}
], lr=0.01, momentum=0.9)

In [89]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations for the training dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),                                                  # Resize images to match ResNet input size
    transforms.ToTensor(),                                                          # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])     # Normalize using ImageNet stats
])

# Load the training dataset
model.load_state_dict(torch.load('../models/best_model.pth', map_location=device))



# Create the DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [90]:
def check_early_stopping(current_acc, best_acc, patience_counter, patience, model, save_path='../models/best_resnet_model.pth'):
    """
    Check if early stopping criteria are met and save the best model if needed.
    """
    should_stop = False
    
    if current_acc > best_acc:
        best_acc = current_acc
        patience_counter = 0
        # Save weights
        torch.save(model.state_dict(), save_path)
        print("Accuracy improved — saving best model.")
    else:
        patience_counter += 1
        print(f"No improvement in accuracy. Patience: {patience_counter}/{patience}")
        
        if patience_counter >= patience:
            print("Early stopping triggered!")
            should_stop = True
            
    return best_acc, patience_counter, should_stop

In [91]:


# Initialize early stopping parameters
best_val_acc = 0.0
patience = 3 
patience_counter = 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
num_epochs = 20  # Increased from 5 to give early stopping a chance to be triggered
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track stats
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Accuracy: {accuracy:.2f}%")
    
    # Check early stopping
    best_val_acc, patience_counter, should_stop = check_early_stopping(
        current_acc=accuracy,
        best_acc=best_val_acc,
        patience_counter=patience_counter,
        patience=patience,
        model=model
    )
    
    if should_stop:
        print(f"Training stopped early at epoch {epoch+1}")
        break

print("Training completed!")

ValueError: not enough values to unpack (expected 2, got 0)